In [ ]:
import pandas as pd
import numpy as np
import fastcore
import math
from tsai.all import *

In [ ]:
lengths = pd.read_csv('../tmp/regime_data_lengths.csv')
regime_data = pd.read_csv('../tmp/regime_data.csv')

In [ ]:
regime_data['duration'] /= 100000000
regime_data['volume'] /= 2000
regime_data['delta'] /= 0.008<
regime_data

In [ ]:
regime_data_raw = regime_data.to_numpy()[:, 2:]
regime_data_deltas = regime_data.to_numpy()[:, 2]
print(regime_data_raw.shape, regime_data_deltas.shape)

In [ ]:
n_history = 20
n_future = 20
step_length = 1
n_samples = (regime_data.shape[0] - (n_history + n_future)) // step_length
print(n_samples)
X = np.empty((n_samples, regime_data.shape[1] - 2, n_history))
y = np.empty((n_samples, regime_data.shape[1] - 2, n_future))
for sample_idx in range(n_samples):
    X[sample_idx] = regime_data_raw[sample_idx * step_length:sample_idx * step_length + n_history, :].transpose()
    y[sample_idx] = regime_data_raw[sample_idx * step_length + n_history:sample_idx * step_length + n_history + n_future, :].transpose()
#y = np.where(y >= 0, 1.0, -1.0)
split_idx = math.floor(n_samples * 0.85)
splits = toL(np.arange(split_idx)), toL(np.arange(split_idx, n_samples))

In [ ]:
X.shape, y.shape, splits

In [ ]:
type(X), type(y), type(splits[0])

In [ ]:
check_data(X, y, splits)

In [ ]:
tfms  = [None, [TSRegression()]]
batch_tfms = TSStandardize(by_sample=True, by_var=True)
dls = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=256)
dls.one_batch()

In [ ]:
dls.show_batch()

In [ ]:
learn = ts_learner(dls, TSTPlus, metrics=[mae, rmse], cbs=ShowGraph())

In [ ]:
learn.lr_find()

In [ ]:
learn.loss_func

In [ ]:
learn.fit_one_cycle(10, 1e-4)

In [ ]:
valid_preds, valid_targets = learn.get_preds(ds_idx=1)
valid_preds.flatten().data, valid_targets.data

In [ ]:
import matplotlib.pyplot as plt
n = 100
#p = np.where(valid_preds.data[n:n+30]*3 >= 0, 1.0, -1.0)
p = valid_preds.data[n:n+100, 4]
t = valid_targets.data[n:n+100, 4]
plt.plot(p, label='Pred')
plt.plot(t, label='Target')
plt.legend()

In [ ]:
learn.save_all('regime_model')

In [ ]:
learn2 = load_learner_all('regime_model')
learn2.freeze()

In [ ]:
targets = []
predictions = []

datas = regime_data.to_numpy()[:, 2:]
datas.shape
for idx in range(1000, 1010):
    frame = datas[idx-n_timesteps:idx].transpose()
    target = datas[idx][0]
    prediction = learn2.get_X_preds(frame)[0][0][0]
    targets.append(target)
    predictions.append(prediction)
targets = np.array(targets)
predictions = np.array(predictions)

import matplotlib.pyplot as plt
n = 50
plt.plot(predictions, label='Pred')
plt.plot(targets, label='Target')
plt.legend()

In [ ]:
regime_data

In [ ]:
idx = 9
a = np.expand_dims(regime_data_raw[idx - n_timesteps + 1:idx + 1, :].transpose(), axis=0)
#a = regime_data_raw[idx - n_timesteps + 1:idx + 1, :].transpose()
a.shape

In [ ]:
result = learn2.get_X_preds(a)

In [ ]:
result[0]

In [ ]:
valid_preds, valid_targets = learn2.get_preds(ds_idx=1)
valid_preds.flatten().data, valid_targets.data

In [ ]:
import matplotlib.pyplot as plt
n = 50
plt.plot(valid_preds.data[:n, 0], label='Pred')
plt.plot(valid_targets.data[:n, 0], label='Target')
plt.legend()